In [16]:
import pandas as pd
import numpy as np
import plotly.express as px

import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import kurtosis
from scipy.stats import skew
from pandas_profiling import ProfileReport

%matplotlib inline

## Get data

In [15]:
print('Retrieving data from git')
url = 'https://raw.githubusercontent.com/bdbritt/weather_flight_delays/main/weather_delay_data_cleaned.csv'
# df = pd.read_csv('weather_delay_data.csv').sample(frac = 0.30)
df = pd.read_csv(url).sample(frac = 0.30)
print(f'Retrieved {df.shape[0]} records')

Retrieving data from git
Retrieved 252009 records


In [9]:
DELAY_COLUMNS = ['ORIGIN', 'NAS_DELAY', 'WEATHER_DELAY']

In [ ]:
# df.memory_usage()

In [ ]:
df.columns

In [ ]:
df.head()

## Main functions

In [33]:
def determine_skewness(df, col):
    """
    Determines skewness of data
    """
    
    print('\n')
    print(df['ORIGIN'].unique()[0])
    
    skew_val = skew(df[col])
    kurtosis_val = kurtosis(df[col])
    
    print(f'skewness value of: {round(skew_val,2)}')
    # if the index is between -1 and 1, then the distribution is symmetric. 
    if -1 <= skew_val <= 1:
        print('data is symmetric')
    # if the index is no more than -1 then it is skewed to the left 
    if skew_val <= -1:
        print('data is left skewed')
        
    # if it is at least 1, then it is skewed to the right
    if skew_val >=1:
        print('data is right skewed')
    
    print(f'\nkurtosis value of: {round(kurtosis_val,2)}')
    # Leptokurtic (Kurtosis > 3): Distribution is longer, tails are fatter. data are heavy-tailed or profusion of outliers
    if kurtosis_val > 3:
        print('distribution is longer and data are heavy-tailed/profusion of outliers')
    
    # Platykurtic: (Kurtosis < 3): Distribution is shorter, tails are thinner than the normal distribution. 
    if kurtosis_val < 3:
        print('distribution is shorter, tails are thinner than normal distribution')
    # data are light-tailed or lack of outliers.

## How many origin records

In [18]:
df['ORIGIN'].value_counts()

ORD    71366
ATL    53270
DFW    49136
DEN    41648
LAX    36589
Name: ORIGIN, dtype: int64

## How many per year

In [19]:
df.groupby('year')['ORIGIN'].value_counts()

year  ORIGIN
2014  ORD       18199
      DFW       13907
      ATL       11540
      DEN       11007
      LAX        7524
2015  ORD       14165
      ATL       10394
      DFW        9700
      DEN        7875
      LAX        7023
2016  ORD        9955
      ATL        8936
      LAX        7930
      DFW        7089
      DEN        6990
2017  ATL       10767
      ORD       10584
      LAX        8009
      DEN        7529
      DFW        6912
2018  ORD       18463
      ATL       11633
      DFW       11528
      DEN        8247
      LAX        6103
Name: ORIGIN, dtype: int64

## How many travel season by origin

In [20]:
org_travel_df = df.groupby('ORIGIN')['travel_season'].value_counts().to_frame('org_travel_season_event_count').reset_index()
org_travel_df

,ORIGIN,travel_season,org_travel_season_event_count
0,ATL,shoulder,22777
1,ATL,high,18120
2,ATL,low,12373
3,DEN,shoulder,17464
4,DEN,high,12221
5,DEN,low,11963
6,DFW,shoulder,22293
7,DFW,high,15026
8,DFW,low,11817
9,LAX,shoulder,16645


## How many year of year

In [21]:
year_org_travel = df.groupby(['year','ORIGIN'])['travel_season'].value_counts().to_frame('count').reset_index()
year_org_travel.head(10)

,year,ORIGIN,travel_season,count
0,2014,ATL,shoulder,5536
1,2014,ATL,high,3150
2,2014,ATL,low,2854
3,2014,DEN,shoulder,4444
4,2014,DEN,low,3624
5,2014,DEN,high,2939
6,2014,DFW,shoulder,6058
7,2014,DFW,high,4441
8,2014,DFW,low,3408
9,2014,LAX,shoulder,3238


## Summary Stats

In [22]:
# how many days with signifant weather delay only
significant_weather_events = df[(df['WEATHER_DELAY'] != 0) & (df['NAS_DELAY']== 0)]
print(f'significant weather events only: {significant_weather_events.shape[0]}')
print(f'percentage: {round(significant_weather_events.shape[0] / df.shape[0] * 100, 2)}')

# how many days with nas delay only
nas_delay_events = df[(df['WEATHER_DELAY'] == 0) & (df['NAS_DELAY'] != 0)]
print(f'\nnas events only: {nas_delay_events.shape[0]}')
print(f'percentage: {round(nas_delay_events.shape[0] / df.shape[0] * 100, 2)}')

# combined events
# TODO think how to handle these results
shared_cnt = df[(df['WEATHER_DELAY'] > 0 ) & (df['NAS_DELAY'] > 0)]
print(f'\ncombined events: {shared_cnt.shape[0]}')
print(f'percentage: {round(shared_cnt.shape[0] / df.shape[0] * 100, 2)}')

significant weather events only: 15751
percentage: 6.25

nas events only: 201659
percentage: 80.02

combined events: 34599
percentage: 13.73


In [ ]:
# shared_cnt.head()

In [23]:
# By origin
df.groupby('ORIGIN')[DELAY_COLUMNS].describe()

NAS_DELAY                                                      \
           count       mean        std  min  25%   50%   75%     max   
ORIGIN                                                                 
ATL      53270.0  22.364764  35.564871  0.0  4.0  13.0  26.0  1142.0   
DEN      41648.0  23.991044  30.692086  0.0  7.0  17.0  29.0   903.0   
DFW      49136.0  19.781158  27.182667  0.0  4.0  13.0  24.0   998.0   
LAX      36589.0  23.269999  32.770002  0.0  6.0  15.0  26.0  1024.0   
ORD      71366.0  24.257994  33.232806  0.0  6.0  16.0  29.0  1203.0   

       WEATHER_DELAY                                                    
               count       mean        std  min  25%  50%  75%     max  
ORIGIN                                                                  
ATL          53270.0  12.720987  52.195607  0.0  0.0  0.0  2.0  1587.0  
DEN          41648.0   6.053448  30.842443  0.0  0.0  0.0  0.0  1070.0  
DFW          49136.0  10.588855  37.149031  0.0  0.0  0.0  0.0  1175.0  
LAX          36589.0   2.223346  18.400592  0.0  0.0  0.0  0.0  1017.0  
ORD          71366.0   9.977608  37.295359  0.0  0.0  0.0  0.0  1466.0

In [24]:
# By origin and travel season
df.groupby(['ORIGIN', 'travel_season'])[DELAY_COLUMNS].describe()

NAS_DELAY                                              \
                         count       mean        std  min  25%   50%   75%   
ORIGIN travel_season                                                         
ATL    high            18120.0  24.255298  37.970329  0.0  3.0  15.0  29.0   
       low             12373.0  20.184272  32.373515  0.0  3.0  11.0  24.0   
       shoulder        22777.0  22.045265  35.163896  0.0  4.0  14.0  25.0   
DEN    high            12221.0  24.299075  32.174735  0.0  6.0  16.0  30.0   
       low             11963.0  23.097718  29.265298  0.0  8.0  17.0  28.0   
       shoulder        17464.0  24.387426  30.573119  0.0  7.0  17.0  29.0   
DFW    high            15026.0  19.425729  27.414374  0.0  4.0  12.0  24.0   
       low             11817.0  20.183888  26.935263  0.0  5.0  13.0  24.0   
       shoulder        22293.0  19.807249  27.154413  0.0  4.0  13.0  24.0   
LAX    high            10937.0  23.946238  32.583827  0.0  6.0  16.0  27.0   
       low              9007.0  22.658155  33.092919  0.0  5.0  15.0  25.0   
       shoulder        16645.0  23.156744  32.710434  0.0  6.0  15.0  25.0   
ORD    high            21414.0  26.414822  37.255866  0.0  7.0  17.0  31.0   
       low             20213.0  22.169198  28.096815  0.0  5.0  16.0  28.0   
       shoulder        29739.0  24.124651  33.287461  0.0  7.0  16.0  28.0   

                             WEATHER_DELAY                                  \
                         max         count       mean        std  min  25%   
ORIGIN travel_season                                                         
ATL    high           1142.0       18120.0  12.649227  43.101109  0.0  0.0   
       low             630.0       12373.0  17.507799  65.143927  0.0  0.0   
       shoulder       1045.0       22777.0  10.177767  50.693703  0.0  0.0   
DEN    high            874.0       12221.0   6.427215  28.943034  0.0  0.0   
       low             791.0       11963.0   6.444621  32.452061  0.0  0.0   
       shoulder        903.0       17464.0   5.523935  30.992881  0.0  0.0   
DFW    high            892.0       15026.0  10.668841  32.544523  0.0  0.0   
       low             793.0       11817.0  10.029449  42.376360  0.0  0.0   
       shoulder        998.0       22293.0  10.831472  37.093000  0.0  0.0   
LAX    high           1024.0       10937.0   1.730731  17.335548  0.0  0.0   
       low             391.0        9007.0   3.539358  22.341717  0.0  0.0   
       shoulder        718.0       16645.0   1.834905  16.598153  0.0  0.0   
ORD    high           1203.0       21414.0   8.895722  35.860940  0.0  0.0   
       low             548.0       20213.0  13.854005  40.465253  0.0  0.0   
       shoulder        967.0       29739.0   8.121927  35.840815  0.0  0.0   

                                         
                      50%   75%     max  
ORIGIN travel_season                     
ATL    high           0.0   8.0   978.0  
       low            0.0   7.0  1587.0  
       shoulder       0.0   0.0  1206.0  
DEN    high           0.0   0.0  1070.0  
       low            0.0   0.0  1050.0  
       shoulder       0.0   0.0  1046.0  
DFW    high           0.0   0.0   798.0  
       low            0.0   0.0  1175.0  
       shoulder       0.0   0.0  1065.0  
LAX    high           0.0   0.0  1017.0  
       low            0.0   0.0   629.0  
       shoulder       0.0   0.0   921.0  
ORD    high           0.0   0.0  1092.0  
       low            0.0  14.0   935.0  
       shoulder       0.0   0.0  1466.0

## Records > 0

In [25]:
# let's see how things look with with records above 0

# separate the two delays
weather_delay = df.loc[df['WEATHER_DELAY'] > 0].copy()
weather_delay = weather_delay[['ORIGIN', 'WEATHER_DELAY', 'travel_season', 'month','year']]
nas_delay = df.loc[df['NAS_DELAY'] > 0].copy()
nas_delay = nas_delay[['ORIGIN', 'NAS_DELAY', 'travel_season', 'month', 'year']]

## How many records above 0?

In [26]:
# weather delay with origin and travel season
weather_delay.groupby(['ORIGIN', 'travel_season'])['WEATHER_DELAY'].describe()

count       mean         std  min   25%   50%    75%  \
ORIGIN travel_season                                                          
ATL    high           5683.0  40.331515   69.334440  1.0  10.0  21.0  44.00   
       low            3782.0  57.277631  107.738581  1.0  10.0  21.0  56.00   
       shoulder       4475.0  51.803128  104.525866  1.0   8.0  19.0  47.00   
DEN    high           1842.0  42.642237   63.366120  1.0  13.0  27.0  51.75   
       low            1837.0  41.968971   73.278334  1.0  10.0  22.0  47.00   
       shoulder       2121.0  45.483263   78.064317  1.0  11.0  23.0  50.00   
DFW    high           3592.0  44.629733   53.994950  1.0  11.0  26.0  61.25   
       low            2472.0  47.944175   82.271167  1.0   9.0  22.0  55.00   
       shoulder       5313.0  45.448146   64.810608  1.0  10.0  25.0  57.00   
LAX    high            425.0  44.538824   76.420288  1.0  12.0  23.0  48.00   
       low             696.0  45.803161   67.301940  1.0  12.0  23.0  51.25   
       shoulder        802.0  38.082294   65.897554  1.0  11.0  20.0  40.75   
ORD    high           3924.0  48.545617   71.373119  1.0  12.0  28.0  58.00   
       low            7832.0  35.754724   58.678015  1.0  10.0  20.0  37.00   
       shoulder       5554.0  43.489017   73.081277  1.0  10.0  21.0  47.00   

                         max  
ORIGIN travel_season          
ATL    high            978.0  
       low            1587.0  
       shoulder       1206.0  
DEN    high           1070.0  
       low            1050.0  
       shoulder       1046.0  
DFW    high            798.0  
       low            1175.0  
       shoulder       1065.0  
LAX    high           1017.0  
       low             629.0  
       shoulder        921.0  
ORD    high           1092.0  
       low             935.0  
       shoulder       1466.0

In [27]:
nas_delay.groupby(['ORIGIN', 'travel_season'])['NAS_DELAY'].describe()

count       mean        std  min  25%   50%   75%  \
ORIGIN travel_season                                                        
ATL    high           15585.0  28.200577  39.560129  1.0  7.0  17.0  33.0   
       low            10784.0  23.158383  33.669036  1.0  5.0  15.0  27.0   
       shoulder       20829.0  24.107014  36.089359  1.0  6.0  15.0  27.0   
DEN    high           11674.0  25.437639  32.477055  1.0  7.0  17.0  31.0   
       low            11744.0  23.528440  29.364860  1.0  8.0  17.0  28.0   
       shoulder       17102.0  24.903637  30.686246  1.0  8.0  17.0  30.0   
DFW    high           13880.0  21.029611  27.926195  1.0  5.0  14.0  25.0   
       low            11325.0  21.060751  27.176478  1.0  6.0  14.0  25.0   
       shoulder       20687.0  21.344951  27.600453  1.0  6.0  15.0  25.0   
LAX    high           10770.0  24.317549  32.697705  1.0  6.0  16.0  28.0   
       low             8722.0  23.398532  33.370713  1.0  6.0  15.0  25.0   
       shoulder       16294.0  23.655579  32.881939  1.0  6.0  15.0  26.0   
ORD    high           20446.0  27.665411  37.671166  1.0  8.0  18.0  33.0   
       low            18081.0  24.783253  28.596024  1.0  8.0  18.0  30.0   
       shoulder       28335.0  25.320028  33.655506  1.0  8.0  17.0  29.0   

                         max  
ORIGIN travel_season          
ATL    high           1142.0  
       low             630.0  
       shoulder       1045.0  
DEN    high            874.0  
       low             791.0  
       shoulder        903.0  
DFW    high            892.0  
       low             793.0  
       shoulder        998.0  
LAX    high           1024.0  
       low             391.0  
       shoulder        718.0  
ORD    high           1203.0  
       low             548.0  
       shoulder        967.0

In [45]:
weather_annual_sum = weather_delay.groupby(['ORIGIN','year'])['WEATHER_DELAY'].sum().to_frame('total_min').reset_index()
weather_monthly_sum = weather_delay.groupby(['ORIGIN','year', 'month'])['WEATHER_DELAY'].sum().to_frame('total_min').reset_index()

nas_annual_sum = nas_delay.groupby(['ORIGIN','year'])['NAS_DELAY'].sum().to_frame('total_min').reset_index()
nas_monthly_sum = nas_delay.groupby(['ORIGIN','year', 'month'])['NAS_DELAY'].sum().to_frame('total_min').reset_index()

## Annual weather sum comparison

In [46]:
fig = px.line(weather_annual_sum, x='year', y='total_min', color='ORIGIN',markers=True)
fig.update_xaxes(type='category')
fig.show()

## Montly sum by Origin and year

In [47]:
# fill in origin as needed
orgin = 'ORD'
weather_filtered_df = weather_monthly_sum.loc[weather_monthly_sum['ORIGIN']==orgin].copy()
fig = px.line(weather_filtered_df, x='month', y='total_min', color='year',markers=True, title=f'Origin: {orgin} Monthly Sum by Year')
fig.show()

## Annual nas sum comparison

In [48]:
fig = px.line(nas_annual_sum, x='year', y='total_min', color='ORIGIN',markers=True)
fig.update_xaxes(type='category')
fig.show()

In [50]:
nas_filtered_df = nas_monthly_sum.loc[nas_monthly_sum['ORIGIN']==orgin].copy()
fig = px.line(nas_filtered_df, x='month', y='total_min', color='year',markers=True,title=f'NAS Delay Origin: {orgin} Monthly Sum by Year')
fig.show()

## Graph Distribution

## Is the data skewed?

In [34]:
weather_delay.groupby('ORIGIN').apply(determine_skewness, 'WEATHER_DELAY')



ATL
skewness value of: 5.61
data is right skewed

kurtosis value of: 43.8
distribution is longer and data are heavy-tailed/profusion of outliers


DEN
skewness value of: 7.2
data is right skewed

kurtosis value of: 75.41
distribution is longer and data are heavy-tailed/profusion of outliers


DFW
skewness value of: 5.71
data is right skewed

kurtosis value of: 59.27
distribution is longer and data are heavy-tailed/profusion of outliers


LAX
skewness value of: 6.24
data is right skewed

kurtosis value of: 60.18
distribution is longer and data are heavy-tailed/profusion of outliers


ORD
skewness value of: 6.54
data is right skewed

kurtosis value of: 69.49
distribution is longer and data are heavy-tailed/profusion of outliers


""


In [35]:
nas_delay.groupby('ORIGIN').apply(determine_skewness, 'NAS_DELAY')



ATL
skewness value of: 6.46
data is right skewed

kurtosis value of: 89.5
distribution is longer and data are heavy-tailed/profusion of outliers


DEN
skewness value of: 6.02
data is right skewed

kurtosis value of: 87.24
distribution is longer and data are heavy-tailed/profusion of outliers


DFW
skewness value of: 5.99
data is right skewed

kurtosis value of: 100.71
distribution is longer and data are heavy-tailed/profusion of outliers


LAX
skewness value of: 4.92
data is right skewed

kurtosis value of: 53.18
distribution is longer and data are heavy-tailed/profusion of outliers


ORD
skewness value of: 7.1
data is right skewed

kurtosis value of: 123.55
distribution is longer and data are heavy-tailed/profusion of outliers


""


In [39]:
origin = 'ATL'
fig = px.histogram(weather_delay.loc[weather_delay['ORIGIN']==origin], x='WEATHER_DELAY', nbins=10, title=f'Histogram of {origin} Weather Delay') # nbins=10 to bin data
# , marginal='rug'
fig.update_layout(yaxis_title="Count")
# fig.add_scatter(x=[2754.20]*100, y=np.linspace(0,400,100),name='mean', mode='lines+markers', hovertext='test', text='text')
fig.show()

In [41]:
fig = px.histogram(nas_delay.loc[nas_delay['ORIGIN']==origin], x='NAS_DELAY', nbins=10, title=f'Histogram of {origin} NAS Delay') # nbins=10 to bin data
# , marginal='rug'
fig.update_layout(yaxis_title="Count")
fig.show()

## Testing

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
def run_isf_model(X: np.array, outliers_fraction = .1) -> IsolationForest: 
    """
    """
    
    clf = IsolationForest(contamination = outliers_fraction, 
                          random_state=42, n_jobs=-1).fit(X)
    return clf


def update_data(df, model, col = 'WEATHER_DELAY') ->pd.DataFrame:
    
    df['scores'] = model.decision_function(df[[col]].values)
    df['anomaly'] = model.predict(df[[col]].values)
    df['anomaly'] = df['anomaly'].apply(lambda x: True if x ==-1 else False)
    
    df.shape[0] - df['anomaly'].value_counts()[0]
    
    return df


def get_model_results(df: pd.DataFrame, col_index=1):
    
    df = df.copy()
    
    # 2d array of column values
    
    X = df.iloc[:, [col_index]].values
    
    model = run_isf_model(X)
    
    df = update_data(df, model)
    
    return df


def run_model(df, group_id = 'ORIGIN') -> pd.DataFrame:
    
    df = df.copy()
    groups = df.groupby(group_id)
    data = pd.concat([get_model_results(v) for k, v in groups])
    
    return data


def quick_stats(df):
    
    record_cnt = df.shape[0]
    anomaly_cnt = record_cnt - df['anomaly'].value_counts()[0]
    
    print(f'group: {df["ORIGIN"].unique()}')
    print(f'total data: {record_cnt}')
    print(f'anomalies detected: {anomaly_cnt}')
    print(f'anomalies %: {anomaly_cnt / record_cnt * 100}')
    print(f'average ISF score: {np.mean(data["scores"])}')
    print(f'non_anomalies detected: {df["anomaly"].value_counts()[0]}')

In [ ]:
data = run_model(weather_delay)

In [ ]:
data.head()

In [ ]:
data.groupby('ORIGIN')['anomaly'].value_counts()

In [ ]:
anomalies = data.loc[data['anomaly']==True].copy()
anomalies['WEATHER_DELAY'].hist(by=anomalies['ORIGIN'], figsize=(10, 10), bins=10, alpha = 0.75, color = 'y')
plt.tight_layout()
plt.show()

In [ ]:
non_anomalies = data.loc[data['anomaly']==False].copy()
non_anomalies['NAS_DELAY'].hist(by=non_anomalies['ORIGIN'], figsize=(10, 10), bins=10, alpha = 0.75, color = 'y')
plt.tight_layout()
plt.show()

In [ ]:
quick_stats(data)

In [ ]:
record_cnt = 

In [ ]:
data.head()

In [ ]:
!pip install kmodes

In [ ]:
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
weather_delay.head()

In [ ]:
test_df = weather_delay.loc[weather_delay['ORIGIN']=='DEN'].copy()
test_df = test_df[['WEATHER_DELAY','travel_season']]
data = test_df.set_index('travel_season')
data.head()

In [ ]:
cost = []
K = range(1,5)
for num_clusters in list(K):
    kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
    kmode.fit_predict(data)
    cost.append(kmode.cost_)
    
plt.plot(K, cost, 'bx-')
plt.xlabel('No. of clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# Building the model with 3 clusters
kmode = KModes(n_clusters=3, init = "random", n_init = 5, verbose=1)
clusters = kmode.fit_predict(data)
clusters

In [ ]:
help(kmode)

In [ ]:
data.insert(0, "Cluster", clusters, True)

In [ ]:
data.head()

In [ ]:
data['Cluster'].value_counts()

In [ ]:
centroids = kmode.cluster_centroids_
centroids

In [ ]:
# fig = px.scatter(nas_delay, x='month', y='NAS_DELAY', color="ORIGIN", symbol="ORIGIN",
#                  facet_col="travel_season")
# fig.show()

In [ ]:
# weather_profile = ProfileReport(weather_delay, title="Weather Delay Report")
# weather_profile.to_notebook_iframe()